In [22]:
from rich import print
import pandas as pd
import numpy as np
import plotly.express as px


In [23]:
rides = pd.read_csv('rides_go.csv')
subscriptions = pd.read_csv('subscriptions_go.csv')
users = pd.read_csv('users_go.csv')

In [24]:
rides['date'] = pd.to_datetime(rides['date'])
rides.head()

,user_id,distance,duration,date
0,1,4409.919140,25.599769,2021-01-01
1,1,2617.592153,15.816871,2021-01-18
2,1,754.159807,6.232113,2021-04-20
3,1,2694.783254,18.511000,2021-08-11
4,1,4028.687306,26.265803,2021-08-28


In [25]:
data = users.merge(rides, on='user_id')
data['date_month'] = data['date'].dt.month
data.head()

,user_id,name,age,city,subscription_type,distance,duration,date,date_month
0,1,Кира,22,Тюмень,ultra,4409.919140,25.599769,2021-01-01,1
1,1,Кира,22,Тюмень,ultra,2617.592153,15.816871,2021-01-18,1
2,1,Кира,22,Тюмень,ultra,754.159807,6.232113,2021-04-20,4
3,1,Кира,22,Тюмень,ultra,2694.783254,18.511000,2021-08-11,8
4,1,Кира,22,Тюмень,ultra,4028.687306,26.265803,2021-08-28,8


In [26]:
px.histogram(data, x='distance', color='subscription_type')

In [27]:
# cd - ceiled duration
data_cd = data
data_cd.duration = data_cd.duration.apply(np.ceil)

In [28]:
users_pivot = pd.pivot_table(
    data_cd, 
    index=['user_id', 'date_month'],
    values=['distance', 'duration'], 
    aggfunc={
        'distance': 'sum',
        'user_id': 'count',
        'duration': 'sum',
    }
)
users_pivot.columns = ['month_distance', 'month_duration', 'month_rides']
users_pivot['subscription_type'] = \
    users_pivot.apply(lambda x: users[users.user_id == x.name[0]].iloc[0][-1], axis=1)
users_pivot.head(10)

C:\Users\User\AppData\Local\Temp\ipykernel_18516\2092732330.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



month_distance  month_duration  month_rides  \
user_id date_month                                                
1       1              7027.511294            42.0            2   
        4               754.159807             7.0            1   
        8              6723.470560            46.0            2   
        10             5809.911100            32.0            2   
        11             7003.499363            56.0            3   
        12             6751.629942            28.0            2   
2       3             10187.723006            63.0            3   
        4              6164.381824            40.0            2   
        6              3255.338202            14.0            1   
        7              6780.722964            48.0            2   

                   subscription_type  
user_id date_month                    
1       1                      ultra  
        4                      ultra  
        8                      ultra  
        10                     ultra  
        11                     ultra  
        12                     ultra  
2       3                      ultra  
        4                      ultra  
        6                      ultra  
        7                      ultra

In [29]:
import warnings

def calculate_month_income(series: pd.Series) -> int:
    user_id: int = series.name[0]
    duration: int = np.ceil(series.month_duration)
    rides_count: int = series.month_rides
    subscription_type: str = series.subscription_type

    type_prices = subscriptions[subscriptions.subscription_type == subscription_type]
    
    total_income: int = int(type_prices.subscription_fee)
    total_income += int(rides_count * type_prices.start_ride_price)
    total_income += int(duration * type_prices.minute_price)
    return total_income
    

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    users_pivot['month_income'] = users_pivot.apply(calculate_month_income, axis=1)


In [30]:
from scipy import stats as st

## Тесты

#### Будет ли помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки?
$H_0$: помесячная выручка от пользователей ultra такая же как и у пользователей free <br>
$H_1$: помесячная выручка от пользователей ultra выше, чем выручка от пользователей free

In [41]:
free = users_pivot.query('subscription_type == "free"')
ultra = users_pivot.query('subscription_type == "ultra"') 
alpha = 0.05

results = st.ttest_ind(
    ultra.month_income,
    free.month_income,
    alternative='greater'
)

print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

p-value: 4.227438894541133e-29

Отвергаем нулевую гипотезу

Это видно и по среднему значению

In [42]:
print(F'{free.month_income.mean()}\n{ultra.month_income.mean()}')

332.89438070020594
367.0291197882197

#### Среднее количество поездок в месяц пользователей с подпиской больше, чем у пользователей без подписки?
$H_0$: Среднее количество поездок в месяц пользователей ultra такая же как и у пользователей free <br>
$H_1$: Среднее количество поездок в месяц пользователей ultra больше, чем у пользователей free

In [32]:
alpha = 0.05
free_month = free['month_rides']
ultra_month = ultra['month_rides']

results = st.ttest_ind(
    ultra_month,
    free_month,
    alternative='greater'
)

print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

p-value: 1.0

Нет оснований отвергнуть нулевую гипотезу

Средние значения очень близки друг к другу

In [43]:
print(F'{ultra_month.mean()}\n{free_month.mean()}')

1.471872931833223
1.723448072962636

#### Средняя дистанция поездки в "холодные" месяца (с октября по март) отличается от "тёплых"?
$H_0$: Средняя дистанция поездки в "холодные" месяца (с октября по март) такая же как в "тёплые" <br>
$H_1$: Средняя дистанция поездки в "холодные" месяца (с октября по март) отличается от "тёплых"

In [54]:
alpha = 0.05
cold_months_data = users_pivot.query('date_month in [10, 11, 12, 1, 2, 3]')
warm_months_data = users_pivot.query('date_month in [4, 5, 6, 7, 8, 9]')

results = st.ttest_ind(
    cold_months_data.month_distance,
    warm_months_data.month_distance,
    alternative='two-sided'
)

print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

p-value: 0.19365908135594664

Нет оснований отвергнуть нулевую гипотезу

Средние значения находятся достаточно близки друг к другу

In [59]:
print(cold_months_data.month_distance.mean())
print(warm_months_data.month_distance.mean())

4945.764339316236

5020.023356959626

#### Среднее время езды на самокате - 30 минут?
$H_0$:  Среднее время поездки равно 30 минут <br>
$H_1$: Средняя время поездки не равно 30 минут

In [65]:
alpha = 0.05
target_value = 30

results = st.ttest_1samp(
    data.duration,
    target_value,
    alternative='two-sided'
)

print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

p-value: 0.0

Отвергаем нулевую гипотезу

Среднее значение != 30

In [66]:
data.duration.mean()

18.30530780944094